# Capacitated Vehicle Routing Problem

## TODO:
- Some explanation and general info of the capacitated vehicle routing problem here. (Will be inserted later)
- Add external file content to notebook
- consider difference between cartesian and geographical coordinates (latitude and longitude are actually angle coordinates, not location coordinates)
- shorten notebook
- display dataframe with .head()
- add time window constraint

First, we import some useful python packages for solving the capacitated vehicle routing problem.

In [1]:
from IPython.display import display, HTML
import openrouteservice
import math
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import folium
from shapely import wkt, geometry
import json
from pprint import pprint
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

# Setting up the problem

In [13]:
CVRP = np.array([[  8.683448,  49.416961,   0  ],
                 [  8.681247,  49.412388,  22  ],
                 [  8.662789,  49.37446,    7  ],
                 [  8.663488,  49.390504,  22  ],
                 [  8.650365,  49.424738,   8  ],
                 [  8.684574,  49.393819,  17  ],
                 [  8.636446,  49.430018,   8  ],
                 [  8.693395,  49.401669,  25  ],
                 [  8.685109,  49.387783,  17  ],
                 [  8.670162,  49.406377,   4  ],
                 [  8.63245,   49.409997,  24  ],
                 [  8.689162,  49.372064,  23  ],
                 [  8.670682,  49.396678,   3  ],
                 [  8.690306,  49.372096,  26  ],
                 [  8.69358,   49.378182,  13  ],
                 [  8.63606,   49.395647,   4  ],
                 [  8.670398,  49.408443,  28  ],
                 [  8.637578,  49.429881,  25  ],
                 [  8.682342,  49.403657,   3  ],
                 [  8.64455,   49.413006,   3  ],
                 [  8.685483,  49.406815,  16  ],
                 [  8.691885,  49.372719,  12  ]])

df_CVRP = pd.DataFrame()

# Solving the capacitated vehicle routing problem

To solve the capacitated vehicle routing problem, we will use the [**google or-tools**](https://github.com/google/or-tools) as well as the [**openrouteservie**](https://openrouteservice.org/). First, we use the matrix API of the openrouteservice, which will give us a symmetric distance or duration matrix for a list of locations/coordinates, where every location is paired with each other. For further explaination please refer to the [**documentation**](https://openrouteservice.org/documentation/#/reference/matrix/matrix). 

We start with querying a distance matrix from the API. The values of the matrix are given in meters.  

Note: If you skipped the previous chapter "Setting up the problem", please make sure that you have executed all previous cells in order for the following code to run without complications. 

In [15]:
response = client.distance_matrix(locations=df_CVRP.iloc[:, :2].as_matrix().tolist(), metrics=['distance'])
dist_matrix = np.array(response['distances'])
df_dist_matrix = pd.DataFrame(dist_matrix)
display(df_dist_matrix.head())

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.00,635.71,7053.42,3983.68,4450.51,3524.03,7753.18,3058.51,3996.68,2046.35,...,3184.58,5916.88,5614.71,6913.38,1935.96,6168.99,2349.44,4934.98,2218.59,6091.06
1,635.71,0.00,6564.96,3495.22,3962.05,3035.57,10709.99,3223.62,3508.22,1557.89,...,2696.12,5428.42,5126.25,6424.92,1447.50,5680.53,1860.98,4446.52,1730.14,5602.60
2,6699.63,6211.17,0.00,4476.63,8260.93,5561.81,15035.00,5838.20,5241.38,5661.26,...,4173.81,4067.51,4829.56,5635.09,5746.39,8454.55,4716.79,9640.36,5150.87,4241.69
3,4470.10,3981.64,3069.76,0.00,6031.41,3332.29,14852.28,3608.67,3804.93,3431.73,...,1944.28,6116.90,5422.96,4796.51,3516.86,6159.43,2487.26,4170.89,2921.34,6291.08
4,4637.54,4149.08,8679.05,5609.31,0.00,5149.67,9166.63,5337.72,5622.32,3040.66,...,4810.22,7542.52,7240.35,4227.68,2653.41,1415.90,3975.07,2249.28,3844.23,7716.70


As we can see the openrouteservice API returns a symmetric distance matrix with the diagonal being zeros, since the distance of every node to itself is zero. In the next step, we use the or-tool on the obtained distance matrix. For this purpose, we first initialize some variables and declare the or-tool solver:

In [16]:
locations = df_CVRP.iloc[:, :2].as_matrix()
demands = df_CVRP.iloc[:, 2].as_matrix()
num_locations = len(locations)
depot = 0    # The depot is the start and end point of each route.
num_vehicles = 4

# Create routing model.
routing = pywrapcp.RoutingModel(num_locations, num_vehicles, depot)
search_parameters = pywrapcp.RoutingModel.DefaultModelParameters()

The SetArcCostEvaluatorOfAllVehicles() method of the or-tool requires a callable object. Therefore we have to wrap the distance matrix and the demands within a class with suitable methods that can be called by the SetArcCostEvaluatorOfAllVehicles() method. 

In [17]:
class matrix_obj(object):
    def __init__(self, matrix):
        self.matrix = matrix
    def Distance(self, from_node, to_node):
        return self.matrix[from_node][to_node]
    
class demand_obj(object):
    def __init__(self, demands):
        self.demands = demands
    def Demand(self, from_node, to_node):
        return self.demands[from_node]
    
# distance callback
dist_matrix_obj = matrix_obj(dist_matrix)
dist_callback = dist_matrix_obj.Distance
routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)

# demand callback
demands_obj = demand_obj(demands)
demands_callback = demands_obj.Demand

Routing problems involve quantities that accumulate along a vehicle's route. The routing solver stores each quantity of this type in an object called a dimension. We now need to add the demand as a new dimension to the routing model. We can do this with the AddDimension() method by providing the demand callback and the vehicle capacity, which is 100 in our case. For every location where a vehicle stops along its route, the total demand on the vehicle increases by the demand at that location.  

In [18]:
routing.AddDimension(demands_callback, 0, 100, True, "Demand")

True

The last step is to solve the routing model and display the solution:

In [ ]:
assignment = routing.SolveWithParameters(search_parameters) # this line is responsible for jupyter kernel crash, reason unknown

In [ ]:


'''
if assignment:
    print("Total distance of all routes: " + str(assignment.ObjectiveValue()) + "\n")
    for vehicle_nbr in range(num_vehicles):
        index = routing.Start(vehicle_nbr)
        index_next = assignment.Value(routing.NextVar(index))
        route = ''
        route_dist = 0
        route_demand = 0

        while (not routing.IsEnd(index_next)):
            node_index = routing.IndexToNode(index)
            node_index_next = routing.IndexToNode(index_next)
            route += str(node_index) + " -> "
            # Add the distance to the next node.
            route_dist += dist_callback(node_index, node_index_next)
            # Add demand.
            route_demand += demands[node_index_next]
            index = index_next
            index_next = assignment.Value(routing.NextVar(index))

        node_index = routing.IndexToNode(index)
        node_index_next = routing.IndexToNode(index_next)
        route += str(node_index) + " -> " + str(node_index_next)
        route_dist += dist_callback(node_index, node_index_next)
        print("Route for vehicle " + str(vehicle_nbr) + ":\n\n" + route + "\n")
        print("Distance of route " + str(vehicle_nbr) + ": " + str(route_dist))
        print("Demand met by vehicle " + str(vehicle_nbr) + ": " + str(route_demand) + "\n")
else:
    print('No solution found.')
'''

In [ ]:
help(pywrapcp.RoutingModel.SolveWithParameters())

# Adding Time Window Constraint